# Performance profiling

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from gpfa_imputation.imputation import *
from gpfa_imputation.data_preparation import *
from gpfa_imputation.utils import*

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from pyprojroot import here
import matplotlib.pyplot as plt

from gpfa_imputation.utils import cache_disk

In [ ]:
cache_path = here() / ".cache"

## Load Data

take the first 200 rows from the Hainich dataset

In [ ]:
hai_path = Path("FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv")
hai_raw = pd.read_csv(here("data") / hai_path, na_values=["-9999", "-9999.99"], parse_dates=[0, 1], nrows=300)

In [ ]:
meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}

units = {
    'TA': '°C',
    'SW_IN': 'W m-2',
    'LW_IN': 'W m-2',
    'VPD': 'hPa'
}

hai = (hai_raw
       .rename(columns=meteo_vars)
       .set_index("TIMESTAMP_END")
       .loc[:, meteo_vars.values()])
hai

,TA,SW_IN,LW_IN,VPD
TIMESTAMP_END,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102
...,...,...,...,...
2000-01-07 04:00:00,3.48,0.0,325.061,0.065
2000-01-07 04:30:00,3.48,0.0,332.588,0.063
2000-01-07 05:00:00,3.47,0.0,332.588,0.063


###  Computation

makes here all the slow computations and cache them on disk

In [ ]:
reset_seed()
data_r_gaps = GPFADataTest(hai).add_random_missing()
data_c_gaps = GPFADataTest(hai).add_gap(15, meteo_vars.values())

In [ ]:
%prun imp = GPFAImputationExplorer(data_r_gaps.data, latent_dims=3).fit()

  0%|          | 0/100 [00:00<?, ?it/s]

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2183.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/torch/autograd/__init__.py:173: UserWarning: operator() profile_node %840 : int[] = prim::profile_ivalue(%838)
 does not have profile information (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:104.)
  Variable._execution_engine.run_backward(  # Calls into the C++ eng

         743587 function calls (703140 primitive calls) in 625.231 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100  472.080    4.721  472.106    4.721 {method 'run_backward' of 'torch._C._EngineBase' objects}
      100  141.402    1.414  141.718    1.417 gpfa.py:64(forward)
        1   10.069   10.069  625.072  625.072 learner.py:54(train)
      100    0.274    0.003    0.274    0.003 {built-in method torch._C._linalg.linalg_cholesky_ex}
      300    0.171    0.001    0.171    0.001 {method 'exp_' of 'torch._C._TensorBase' objects}
        1    0.154    0.154  625.226  625.226 imputation.py:154(fit)
      400    0.049    0.000    0.064    0.000 {built-in method builtins.sum}
      200    0.046    0.000    0.046    0.000 {method 'contiguous' of 'torch._C._TensorBase' objects}
      100    0.030    0.000    0.030    0.000 {built-in method torch.triangular_solve}
      300    0.024    0.000    0.024    0.000 rbf_cova